In [1]:
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_theme(style="whitegrid")

sys.path.append("../src")
from retriever import FaissComplaintRetriever
from generator import AnswerGenerator
from prompts import RAG_PROMPT
from rag_pipeline import RAGPipeline

### Load FAISS Vector Store

In [2]:
INDEX_PATH = "../vector_store/index.faiss"
METADATA_PATH = "../vector_store/metadata.pkl"

retriever = FaissComplaintRetriever(
    index_path=INDEX_PATH,
    metadata_path=METADATA_PATH
)

print("✓ FAISS vector store loaded")


✓ FAISS vector store loaded


In [3]:
print(retriever.metadata.columns)

Index(['complaint_id', 'product_category', 'chunk_text', 'chunk_index',
       'total_chunks'],
      dtype='object')


### Retrieval Demo

In [4]:
query = "Why are customers unhappy with credit cards?"

documents, metadata = retriever.retrieve(query, k=5)

print(f"Retrieved {len(documents)} documents\n")

for i, meta in enumerate(metadata, 1):
    print(f"Source {i}")
    print(f"Product: {meta['product_category']}")
    print(f"Complaint Chunk: {meta['chunk_text']}")
    print(documents[i-1][:300], "...\n")


Retrieved 2 documents

Source 1
Product: Savings Account
Complaint Chunk:  my account and wondering if more fraudulent charges are going to come through my account and mess up my redacted
 my account and wondering if more fraudulent charges are going to come through my account and mess up my redacted ...

Source 2
Product: Credit Card
Complaint Chunk: tore redacted also charged merchandise on my company accounts receivable account without my consent or approval and in fact signed my name to the sales receipt for that transaction as well see exhibit a comparing my signature on my drivers license see exhibit b and the reciept it is clear that is not my signature although they have corrected the unauthorized use of my company accounts receivable account to date the issue of the fraudently obtained credit using my personal information including b
tore redacted also charged merchandise on my company accounts receivable account without my consent or approval and in fact signed my name to the

### Load LLM

In [5]:
generator = AnswerGenerator()
print("✓ LLM loaded")

Device set to use mps:0


✓ LLM loaded


### Inspect Prompt

In [6]:
print("RAG PROMPT TEMPLATE")
print("=" * 80)
print(RAG_PROMPT)

RAG PROMPT TEMPLATE

You are a financial analyst assistant for CrediTrust Financial.

Based ONLY on the complaint excerpts below, explain the MAIN REASONS
customers are unhappy with the product.

Rules:
- Do NOT quote or paraphrase any single complaint
- Do NOT mention exhibits, signatures, or redactions
- Summarize patterns across multiple complaints
- Write in 2–4 clear, analytical sentences

Complaint Excerpts:
{context}

Question:
{question}

Final Answer:



### Manual Generation

In [7]:
answer = generator.generate(query, documents)

print("Generated Answer")
print("-" * 60)
print(answer)

Generated Answer
------------------------------------------------------------
fraudulent charges are going to come through my account and mess up my redacted tore redacted also charged merchandise on my company accounts receivable account without my consent or approval and in fact signed my name to the sales receipt for that transaction as well see exhibit a comparing my signature on my drivers license see exhibit b and the reciept it is clear that is not my signature although they have corrected the unauthorized use of my company accounts receivable account to date the issue of the fraudently obtained credit using my personal information including b


### Full RAG Pipeline

In [8]:
rag = RAGPipeline(
    index_path=INDEX_PATH,
    metadata_path=METADATA_PATH
)

final_answer, sources = rag.run(query)

print("Final RAG Answer")
print("-" * 60)
print(final_answer)

Device set to use mps:0


Final RAG Answer
------------------------------------------------------------
fraudulent charges are going to come through my account and mess up my redacted tore redacted also charged merchandise on my company accounts receivable account without my consent or approval and in fact signed my name to the sales receipt for that transaction as well see exhibit a comparing my signature on my drivers license see exhibit b and the reciept it is clear that is not my signature although they have corrected the unauthorized use of my company accounts receivable account to date the issue of the fraudently obtained credit using my personal information including b


### Display Sources

In [9]:
print("Sources Used")
print("-" * 60)

for i, src in enumerate(sources, 1):
    print(f"\nSource {i}")
    print(f"Product: {src['product_category']}")
    print(
        f"Complaint ID: {src['complaint_id']} "
        f"(chunk {src['chunk_index'] + 1}/{src['total_chunks']})"
    )
    print(src["text"][:400], "...")



Sources Used
------------------------------------------------------------

Source 1
Product: Savings Account
Complaint ID: None (chunk 5/5)
 my account and wondering if more fraudulent charges are going to come through my account and mess up my redacted ...

Source 2
Product: Credit Card
Complaint ID: None (chunk 6/8)
tore redacted also charged merchandise on my company accounts receivable account without my consent or approval and in fact signed my name to the sales receipt for that transaction as well see exhibit a comparing my signature on my drivers license see exhibit b and the reciept it is clear that is not my signature although they have corrected the unauthorized use of my company accounts receivable a ...


### Multi-Question Evaluation

In [ ]:
questions = [
    "What are the most common complaints about money transfers?",
    "Are there recurring fraud-related issues?",
    "Do personal loan customers complain about interest rates?",
    "What problems are reported for savings accounts?"
]

for q in questions:
    ans, _ = rag.run(q)
    print("\nQUESTION:", q)
    print("ANSWER:", ans)



QUESTION: What are the most common complaints about money transfers?
ANSWER: im a victim of identity theft and this company has refused to follow the laws set out in the fair credit reporting act and block this information from appearing on my credit file to add insult to injury they claim to have verified the information despite them being the one who alerted me to my identity theft in the first place redactedredacted2014 i signed a lease agreement underwritten by chase auto finance my boyfriend let s call him bf had committed to making the payments and i paid the down payment and initial tax title and fees in redacted i logged in to chase s online payment portal for bf to authorize his bank account redacted redacted in order for chase to pull automatic monthly payments from his redacted redacted account the authorization process required bf to log in to his account and verify a small deposit carecredits policy of charging a late fee irrespective of whether a payment is late evidence